### Data Reading

## Load data

1. CSV File

In [0]:
path = '/Volumes/workspace/learning/csvfile/BigMart Sales.csv'

In [0]:
df_csv = spark.read.format('csv')\
    .option('header',True)\
        .option('inferSchema',True)\
            .load(path)

In [0]:
df_csv.display()